# Visualisation example Part 1

This notebook produces a GIF showing the progression of holdsAt() fluents which relate to a particular record over time. A prolog query 
finds all records which have been linked to an organ donation decision before a given time period, and the output from this file is stored in a file. The output is processed to produce a graph of known records at each time period, which are saved as PNGs and concatenated into a GIF using moviepy. 

NB: For unknown reasons, importing moviepy on the Prolog focused kernel is not working. This notebook contains the visualisation set up until moviepy is needed, at which point **VisualisationPart2.ipynb**, which was run in a Python kernel shows the rest of the example!

In [1]:
% Set up required to reset environment
?- cd('~/work'), ['dec'].
?- initialiseDEC.
?- retractall(happensAtNarrative(_,_)).

true.
false.
true.

Below are the definitions for parent child relationships between records. For example, a message with a test result is the child of the request for that test, and a linkEvidence record connects a decision record to a evidence record.

 As state constraints have not yet been implemented, this code currently uses descendant/2 fluents rather than holdsAt(descendant/2) fluents. holdsAt() fluents should replace the existing fluents once dec.pl has been updated appropriately.

In [2]:
% File: depends.pl
descendant(GID,RID):- holdsAt(child(GID,RID),12).
descendant(GID, RID) :- holdsAt(child(GID, MID),12),descendant(MID, RID).

initiates(record(RID,linkEvidence(DID,EID,Actor)), child(RID,DID),_).
initiates(record(RID,linkEvidence(DID,EID,Actor)), child(EID,RID),_).
initiates(record(NID,resultMessage(RID,_,_)), child(RID,NID),_).
initiates(record(NID,message(RID,_,_)), child(RID,NID),_).

The cell below contains basic initiates statements from the wider organ donation example. Exp_rules have not been included here for simplicity. 

In [3]:
% File: PatientCareProcedure.pl

% History requests are not yet fulfilled by default. History grants answer history requests
initiates(record(ID,histRequest(Patient,Source)), record(ID,histRequested(Patient,Source)='NA'), _).
initiates(record(_,message(ID, _, Source)), record(ID,histRequested(Patient,Source)='ANS'),T):- holdsAt(record(ID,histRequested(Patient,Source)='NA'),T).
initiates(record(EID,resultMessage(RID,Res,Lab)), record(EID,resultMessaged(RID,Res,Lab)),T).
% Result Messages answer test requests
initiates(record(ID,testRequest(Patient,Test,Lab,Requester)), record(ID,testRequested(Patient,Test,Lab,Requester)='NA'), _).
initiates(record(_,resultMessage(ID, _, Source)), record(ID,testRequested(_,_,Source,_)='ANS'),T):- holdsAt(record(ID,testRequested,_,_,Source,_)='NA'),T).

% Event to fluent
initiates(record(ID,diagnosis(Patient,Condition,Result,Actor)), record(ID,diagnosed(Patient,Condition,Result,Actor)),_).
initiates(record(LID,linkEvidence(ID,EID,Actor)), record(LID,linkedEvidence(ID,EID,Actor)),_).
initiates(record(RID,donorDecision(PatID,Concl,Actor)), record(RID,donorDecided(PatID,Concl,Actor)),_).
initiates(record(NID,message(RID,M,Source)), record(NID,messaged(RID,M,Source)),_).
%initiates(record(LID,linkEvidence(ID,EID,Actor)), record(LID,linkedResult(ID,EID,Actor)),T):- holdsAt(record(EID,resultMessaged(,_,_,_)),T).
%initiates(record(LID,linkEvidence(ID,EID,Actor)), record(LID,linkedDiagnosis(ID,EID,Actor)),T):- holdsAt(record(EID,diagnosed(_,_,_,_)),T).
initiates(record(ID,brainDeath(PatID,Actor)), record(ID,brainDead(PatID,Actor)),_).


Relevant narrative to model:

In [4]:
% File: narrative.pl

% Test narrative

happensAtNarrative(record(0,admission(124)),1).
happensAtNarrative(record(13,testRequest(123, 'HIV','Otago Blood', 'AH')),2).
happensAtNarrative(record(14,resultMessage(13,positive, 'Otago Blood')),3).
happensAtNarrative(record(11,diagnosis(123,'HIV',positive,'PS')),4).
happensAtNarrative(record(15,linkEvidence(11,14,'PS')),5).
happensAtNarrative(record(10,histRequest(123,'Well')),2).
%                   message(newID, requestID, happensAt or HoldsAt(type(ID,...),time), source), time)
happensAtNarrative(record(12,message(10, happensAt(diagnosis(123,'Hep-C','Positive'),1),'Well')),4).
happensAtNarrative(record(16,brainDeath(123,'PS')),6).
happensAtNarrative(record(17,brainDeath(123,'FG')),7).
happensAtNarrative(record(18,donorDecision(123,yes,'FG')),8).
happensAtNarrative(record(19,donorDecision(126,yes,'FG')),8).
happensAtNarrative(record(20,linkEvidence(18,11,'FG')),9).
happensAtNarrative(record(21,linkEvidence(18,12,'FG')),9).

In [5]:
?- run(12).

true.

Here we define a helper function to concatenate lists, which is used in the findRelationships and findRecords rules. findRelationships\3 finds all child parent relationships for records which are descendants of a specified record, while findRecords\3 finds the records themselves.

In [6]:
list_concat([],L,L).
list_concat([X1|L1],L2,[X1|L3]) :- list_concat(L1,L2,L3).
findRelationships(-1,_,[]).
findRelationships(X,C,L):- X>= 0, Z is X - 1, findRelationships(Z, C,M),   findall([X,Child,Parent], (holdsAt(child(Child,Parent),X),descendant(Child,C)), A), list_concat(A,M,L).
findRecords(-1,_,[]).
findRecords(X,C,L):- X>= 0, Z is X - 1, findRecords(Z, C, M),   findall([X,ID,Contents], (holdsAt(record(ID,Contents),X),(descendant(ID,C); ID = C)), A), list_concat(A,M,L).


We find the relationships and related records for record 18 at time 12 and save the output to a file.

In [7]:
% Output: AnimationOutput.txt
% The query output is saved to the specified file
?- A = 12, findRelationships(A,18,X).
?- findRecords(12,18,X).

A = 12, X = [ [ 12, 13, 14 ], [ 12, 10, 12 ], [ 12, 15, 11 ], [ 12, 14, 15 ], [ 12, 20, 18 ], [ 12, 11, 20 ], [ 12, 21, 18 ], [ 12, 12, 21 ], [ 11, 13, 14 ], [ 11, 10, 12 ], [ 11, 15, 11 ], [ 11, 14, 15 ], [ 11, 20, 18 ], [ 11, 11, 20 ], [ 11, 21, 18 ], [ 11, 12, 21 ], [ 10, 13, 14 ], [ 10, 10, 12 ], [ 10, 15, 11 ], [ 10, 14, 15 ], [ 10, 20, 18 ], [ 10, 11, 20 ], [ 10, 21, 18 ], [ 10, 12, 21 ], [ 9, 13, 14 ], [ 9, 10, 12 ], [ 9, 15, 11 ], [ 9, 14, 15 ], [ 8, 13, 14 ], [ 8, 10, 12 ], [ 8, 15, 11 ], [ 8, 14, 15 ], [ 7, 13, 14 ], [ 7, 10, 12 ], [ 7, 15, 11 ], [ 7, 14, 15 ], [ 6, 13, 14 ], [ 6, 10, 12 ], [ 6, 15, 11 ], [ 6, 14, 15 ], [ 5, 13, 14 ], [ 5, 10, 12 ], [ 4, 13, 14 ] ] .
X = [ [ 12, 13, testRequested(123, HIV, Otago Blood, AH)=NA ], [ 12, 10, histRequested(123, Well)=NA ], [ 12, 14, resultMessaged(13, positive, Otago Blood) ], [ 12, 11, diagnosed(123, HIV, positive, PS) ], [ 12, 10, histRequested(123, Well)=ANS ], [ 12, 12, messaged(10, happensAt(diagnosis(123, Hep-C, Positive), 

We read the saved query output in from the file and save the relationships and records which hold at each time period from 0 up to the maximum (12) in allEdges and allNodes respectively.

In [8]:
%Python
from array import *
# Read in prolog output
f = open("/work/output_files/AnimationOutput.txt", "r")
firstLine = f.readline()
# Remove extra spacing, brackets
relationships = firstLine[16:-7]
records = f.readline()[8:-7]
f.close()
# Save end time period
maxTime = int(firstLine.split(",")[0].split(" = ")[1])
# allEdges stores all the records that hold for each time period
allEdges = []
# allNodes stores all the parent-child relationships that hold for each time period
allNodes = []
for i in range(maxTime+1):
    allEdges.append({})
    allNodes.append({})
relationships = relationships.split(" ], [ ")
records = records.split(" ], [ ")
for rel in relationships:
    time, child, parent = rel.split(", ")
    time = int(time)
    if parent in allEdges[time]:
        allEdges[time][parent].append(child)
    else:
        allEdges[time][parent] = [child]
for rec in records:
    time = int(rec[0: rec.index(',')])
    rec = rec[rec.index(',')+1:]
    ID = rec[1: rec.index(',')]
    contents = rec[rec.index(',')+1:]
    allNodes[time][ID] = contents

In order to create a graph for each time period, we need to import pygraphviz. We add the edges and nodes to a new graph for each time period and save each graph as a PNG (it is also possible to produce SVGs, but there was less support for creating GIFs).

In [9]:
%Python
import pygraphviz as pgv
from graphviz import Digraph
# Produce a PNG grpah for each time period
for i in range(maxTime+1):
    di = Digraph()
    di.graph_attr["label"] = "Time: %s" % i
    di.format = 'png'
    for key, value in allNodes[i].items():
        di.node(key,label=value)
    for key, value in allEdges[i].items():
        for v in value:
            di.edge(key,v)
    if i < 10:
        fname = "VisualisationExample/Timeline/di_0%s" % i
    else:
        fname = "VisualisationExample/Timeline/di_%s" % i
    di.render(filename=fname)

Converting the PNGs to a GIF uses moviepy. 

By default the PNG concatenation method centres PNGs of all sizes into the frame of the largest PNG, which makes the transition between images quite jolting. An alternative concatenate.py file could be used which centres PNGs horizontally but positions them at the bottom of the frame. (Unfortunately, Deepnote had issues with using git clone or other methods to install moviepy and change the contents of the file, so this example does not use the alternate file, although it did work at one point!)

In [10]:
%Python
import os
os.system('pip install moviepy')
#os.system('cp /work/VisualisationExample/concatReplacement.py ~/venv/lib/python3.9/site-packages/moviepy/video/compositing/concatenate.py')


Here the cell (should!) concatenate the PNGs together to produce a GIF. It currently doesn't work in this kernel. See visualisationPart2.ipynb for the functional version.

In [11]:
%Python
import glob
#from moviepy.editor import *
import moviepy.editor as mpv
input_png_list = glob.glob("/work/VisualisationExample/Timeline/*.png")
input_png_list.sort()
clips = [mpv.ImageClip(i).set_duration(2) for i in input_png_list]
for c in clips:
    print(c)
concat_clip = mpv.concatenate_videoclips(clips, method="compose", bg_color=[255,255,255])
concat_clip.write_gif("/work/VisualisationExample/process.gif", fps=2)

ERROR: Script gave error name 'mpv' is not defined

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=1527cc64-36a2-4b35-bd8b-8d493ca554fa' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>